In [1]:
# Importing libraries and preparing the model
import cv2
import numpy as np
from keras.models import load_model
model=load_model("./model-010.model")  # load the model with the best performance

In [2]:
# Set indicators and haarcascade file
indicators={0:'No mask detected',1:'Mask detected!'}
color_dict={0:(0,0,255),1:(0,255,0)}

size = 4
cam = cv2.VideoCapture(0) # Define video capture object

# We load the xml file
classifier = cv2.CascadeClassifier(r"C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\cv2\data\haarcascade_frontalface_default.xml")

# Run the program
while True:
    
    # Capture video frame by frame (infinite while loop)
    (rval, frame) = cam.read()
    frame = cv2.flip(frame, 1, 1) # Flip 2D array (like a mirror)

    # Resize the image to speed up detection
    mini = cv2.resize(frame, (frame.shape[1] // size, frame.shape[0] // size))

    # Detect MultiScale / faces 
    faces = classifier.detectMultiScale(mini)

    # Draw rectangles around each face
    for f in faces:
        (x, y, w, h) = [v * size for v in f] # Scale the shapesize backup
        # Save just the rectangle faces in SubRecFaces
        face_img = frame[y:y+h, x:x+w]
        resized=cv2.resize(face_img,(150,150))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)
        
        label=np.argmax(result,axis=1)[0]
      
        # Defining the label details
        cv2.rectangle(frame,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(frame,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(frame, indicators[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
    # Show the image
    cv2.imshow('LIVE', frame)
    key = cv2.waitKey(10)

    # Break out of the loop when pressing Esc
    if key == 27: # ASCII code for Esc
        break

# Stop video
cam.release()

# Close all started windows
cv2.destroyAllWindows()

In [4]:
# In case LED stays on
cam.release()